## Import

In [4]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from datetime import datetime, timedelta, timezone
from itertools import combinations, product

import gc
import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
!pip install "pystan<2.18.1"

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import pystan

# Device & Path

In [2]:
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
# torch.backends.mps.is_available()

In [8]:
# PATH
DATA_PATH  = '../DATA'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TRAIN_CSV  = os.path.join(DATA_PATH, 'train.csv')
SAMPLE_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')

## Fixed RandomSeed

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True


seed_everything(42) # Seed 고정

## Data Pre-processing

In [10]:
df_train = pd.read_csv(TRAIN_CSV)
df_ss = pd.read_csv(SAMPLE_PATH)
df_train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


## columns 
1. '일자': 이 컬럼은 특정 거래가 발생한 날짜를 나타냅니다. "YYYY-MM-DD" 형식으로 표현됩니다.

2. '종목코드': 각각의 주식을 식별하는 고유한 코드입니다. 한국의 경우 종목코드는 대부분 6자리 숫자로 이루어져 있습니다. (ex - A060310)

3. '종목명': 주식의 공식 이름을 나타냅니다. 이 이름은 주로 회사의 이름을 반영하며, 시장에서 해당 주식을 찾을 때 사용됩니다.

4. '거래량': 특정 일자에 해당 주식이 거래된 총 주식 수를 나타냅니다. 거래량은 시장의 활동 수준과 관심도를 반영하는 중요한 지표입니다.

5. '시가': 주식 시장이 개장했을 때의 첫 거래 가격을 의미합니다. 이는 해당 날의 시장 흐름을 이해하는데 도움이 됩니다.

6. '고가': 특정 일자에 해당 주식이 거래된 가장 높은 가격을 나타냅니다.

7. '저가': 특정 일자에 해당 주식이 거래된 가장 낮은 가격을 나타냅니다.

8. '종가': 주식 시장이 마감했을 때의 마지막 거래 가격을 의미합니다. 종가는 해당 일의 주식 가격 변동을 반영하며, 이후의 시장 분석에 중요한 기준이 됩니다.

In [11]:
# change columns name
df = df_train.rename(columns={
    '일자': 'date',
    '종목코드': 'code',
    '종목명': 'name',
    '거래량': 'volume',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종가': 'close'
})
# datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df_processed = df.copy()
le = preprocessing.LabelEncoder()
df_processed['code'] = le.fit_transform(df_processed['code'])  
df_processed.drop(columns=['name'], inplace=True)


In [12]:
def prep_prices(df):
    
    from decimal import ROUND_HALF_UP, Decimal
    
    pcols = ['open', 'high', 'low', 'close']

    #df.ExpectedDividend.fillna(0,inplace=True) # No column 'ExpectedDividend' in the given data
    
    def qround(x):
        return float(Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP))
    
    def adjust_prices(df_code):
        df_code = df_code.sort_values("date", ascending=False)
    
        # generate adjusted prices
        for p in pcols:     
            df_code.loc[:, p] = df_code[p].apply(qround)
        df_code.loc[:, "volume"] = df_code["volume"]
        df_code.ffill(inplace=True)
        df_code.bfill(inplace=True)
        

        return df_code

    # generate Adjusted
    df = df.sort_values(["code", "date"])
    df = df.groupby("code").apply(adjust_prices).reset_index(drop=True)
    df = df.sort_values("date")
    return df


In [13]:
prep_prices(df_processed)

,date,code,volume,open,high,low,close
493999,2021-06-01,999,33381,172500.0,176900.0,172000.0,175300.0
327027,2021-06-01,661,0,0.0,0.0,0.0,1055.0
327521,2021-06-01,662,421016,13100.0,13450.0,12950.0,13400.0
328015,2021-06-01,663,219380,9000.0,9000.0,8770.0,8840.0
328509,2021-06-01,664,355229,2820.0,2865.0,2770.0,2810.0
...,...,...,...,...,...,...,...
91884,2023-05-30,186,0,0.0,0.0,0.0,4780.0
495976,2023-05-30,1004,15889,5970.0,6100.0,5930.0,5950.0
859560,2023-05-30,1740,86795,17200.0,17890.0,16630.0,16630.0
231686,2023-05-30,469,483434,8770.0,8860.0,8750.0,8760.0


- test를 위한 코드

In [14]:
# Process the data using the given function
df_train = prep_prices(df_processed)

# Select data from 2021-06-03 to 2023-05-30
mask = (df_train['date'] >= "2021-06-03") & (df_train['date'] <= "2023-05-30")
df_train = df_train.loc[mask]

# Prepare the data for Prophet
df_train = df_train.rename(columns={"date": "ds", "close": "y"})

# Separate features by 'code'
data_dict = {}
unique_codes = df_train['code'].unique()

for code in unique_codes:
    data_dict[code] = df_train[df_train['code'] == code][['ds', 'y']]


In [15]:
data_dict[661]

,ds,y
327025,2021-06-03,1055.0
327024,2021-06-04,1055.0
327023,2021-06-07,1055.0
327022,2021-06-08,1055.0
327021,2021-06-09,1055.0
...,...,...
326538,2023-05-23,2110.0
326537,2023-05-24,2110.0
326536,2023-05-25,2110.0
326535,2023-05-26,2110.0


In [16]:
from prophet import Prophet

In [17]:
tt = data_dict[661]
reset = tt.reset_index(drop=True)
reset

,ds,y
0,2021-06-03,1055.0
1,2021-06-04,1055.0
2,2021-06-07,1055.0
3,2021-06-08,1055.0
4,2021-06-09,1055.0
...,...,...
487,2023-05-23,2110.0
488,2023-05-24,2110.0
489,2023-05-25,2110.0
490,2023-05-26,2110.0


In [18]:
Prophet()

In [19]:
Prophet().fit(reset)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/bkn_fjjq.json
DEBUG:cmdstanpy:input tempfile: /var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/4lyfhi3t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=9862', 'data', 'file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/bkn_fjjq.json', 'init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/4lyfhi3t.json', 'output', 'file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/prophet_model_2ppl8tj/prophet_model-20230719223112.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=8976 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/ta8hbnzn.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/8g2_i5z3.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpgpsnd2nv/prophet_modelv_wfihqb/prophet_model-20230719223112.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[9520]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such fil

In [24]:
from prophet import Prophet
model = Prophet()
model.fit(reset)

# # Create a dataframe to hold the future dates
future = model.make_future_dataframe(periods=1)

# Predict on the future dates
forecast = model.predict(future)
predictions_dict = {}
# Save the predictions to the dictionary
predictions_dict[code] = forecast

21:29:58 - cmdstanpy - INFO - Chain [1] start processing
21:29:59 - cmdstanpy - INFO - Chain [1] done processing
21:29:59 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
21:29:59 - cmdstanpy - INFO - Chain [1] start processing
21:29:59 - cmdstanpy - INFO - Chain [1] done processing
21:29:59 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=47393 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpt1t4gk6p/7c1wv301.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpt1t4gk6p/rc7zlpue.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpt1t4gk6p/prophet_modeleyc5tx42/prophet_model-20230719212959.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[7426]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such fi

In [27]:
import psutil

# Get the current memory usage
mem = psutil.virtual_memory()

# Print the total, available, and used memory
print(f"Total memory: {mem.total}")
print(f"Available memory: {mem.available}")
print(f"Used memory: {mem.used}")


Total memory: 8589934592
Available memory: 2907717632
Used memory: 4207837184


In [24]:
!export DYLD_FALLBACK_LIBRARY_PATH=/usr/local/lib:$DYLD_FALLBACK_LIBRARY_PATH


In [20]:
!pip uninstall prophet -y
!pip install prophet

Found existing installation: prophet 1.1.4
Uninstalling prophet-1.1.4:
  Successfully uninstalled prophet-1.1.4
Defaulting to user installation because normal site-packages is not writeable
  Using cached prophet-1.1.4-py3-none-macosx_11_0_universal2.whl


In [51]:
!pip install prophet --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached prophet-1.1.4-py3-none-macosx_11_0_universal2.whl


In [48]:
model.make_future_dataframe(periods=1)

,ds
0,2021-06-03
1,2021-06-04
2,2021-06-07
3,2021-06-08
4,2021-06-09
...,...
488,2023-05-24
489,2023-05-25
490,2023-05-26
491,2023-05-30


In [47]:
data_dict[661]

,ds,y
327025,2021-06-03,1055.0
327024,2021-06-04,1055.0
327023,2021-06-07,1055.0
327022,2021-06-08,1055.0
327021,2021-06-09,1055.0
...,...,...
326538,2023-05-23,2110.0
326537,2023-05-24,2110.0
326536,2023-05-25,2110.0
326535,2023-05-26,2110.0


- 아마 미래기간에 대한 데이터 프레임을 따로 생성해줘야할 듯
- In sample test 진행 후에 out of sample test를 진행한다
- 밑에 코드는 예시코드 

In [130]:
from prophet import Prophet
# Initialize a dictionary to hold prediction dataframes
predictions_dict = {}

# Loop over each code
for code in unique_codes:
    # Initialize Prophet and fit the data
    model = Prophet()
    model.fit(data_dict[code])

    # Create a dataframe to hold the future dates
    future = model.make_future_dataframe(periods=15)

    # Predict on the future dates
    forecast = model.predict(future)

    # Save the predictions to the dictionary
    predictions_dict[code] = forecast

# Concatenate all prediction dataframes
df_predictions = pd.concat(predictions_dict.values())


22:17:47 - cmdstanpy - INFO - Chain [1] start processing
22:17:48 - cmdstanpy - INFO - Chain [1] done processing
22:17:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
22:17:48 - cmdstanpy - INFO - Chain [1] start processing
22:17:48 - cmdstanpy - INFO - Chain [1] done processing
22:17:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=2208 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/gcazj4ay.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/r9piiqrl.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/prophet_modelyztio2l0/prophet_model-20230706221748.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[34605]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such fi

In [189]:
import psutil

# Get the memory details
memory_info = psutil.virtual_memory()

print(f"Total memory: {memory_info.total / (1024.0 ** 3)} GB")
print(f"Available memory: {memory_info.available / (1024.0 ** 3)} GB")
print(f"Used memory: {memory_info.used / (1024.0 ** 3)} GB")
print(f"Memory percent used: {memory_info.percent}%")

Total memory: 8.0 GB
Available memory: 1.6962738037109375 GB
Used memory: 2.98797607421875 GB
Memory percent used: 78.8%


In [245]:
!pip install finance-datareader

176423.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [246]:
!pip install pykrx


176434.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 4.8 MB/s eta 0:00:00a 0:00:01


#### 앞으로 할것 
- validation data 구축 -> 비교 분석
- shape ratio 공식
- prophet / lightgbm / transformer 비교